# Azure AutoML Local Compute Run 
## (Porto Seguro's Safe Driving Prediction)

Now let's use Azure Automated ML!

In this notebook you are going to use a **'Local Compute Run'** which is a better approach when getting started and trying small datasets because it needs less overhead on infrastructure preparation since it'll use your read-to-run Jupyter environment and compute from your machine (versus Azure ML remote runs which require to sping up VMs in the cloud for remote compute).

However, when targeting larger datasets and production trainings, it'll be better to use Azure ML Remote Compute since you will be able to parallelize runs/trainings and scale out on multiple VMs in the Azure ML training cluster. You can see that other approach in the 'automl-remote-compute-run-safe-driver-classifier.ipynb' notebook.

## Import Needed Packages

Import the packages needed for this solution notebook. The most widely used package for machine learning is [scikit-learn](https://scikit-learn.org/stable/), [pandas](https://pandas.pydata.org/docs/getting_started/index.html#getting-started), and [numpy](https://numpy.org/). 

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn import metrics

## Check Azure ML SDK version

In [3]:
import azureml.core
print("This notebook was tested using version 1.6.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was tested using version 1.6.0 of the Azure ML SDK
You are currently using version 1.6.0 of the Azure ML SDK


##  Get Azure ML Workspace to use

In [4]:
from azureml.core import Workspace, Dataset

# Get Workspace defined in by default config.json file
ws = Workspace.from_config()

## Load data from file into regular Pandas DataFrame

In [5]:
# Directly load from file into Pandas DataFrame
DATA_DIR = "../../data/"
data_df = pd.read_csv(os.path.join(DATA_DIR, 'porto_seguro_safe_driver_prediction_train.csv'))

print(data_df.shape)
data_df.head(5)

(595212, 59)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


## Split Data into Train and Test Sets

Partitioning data into training, validation, and holdout/test sets allows you to develop highly accurate models that are relevant to data that you collect in the future, not just the data the model was trained on. 

In this notebook we holdout a test dataset to calculate the metrics with that set "not seen" by AutoML, after the AutoML process is finished.
Not taking into account the test dataset, AutoML will by default internally either use a validation dataset split from the train dataset or use cross-validation, depending on the size of the data (larger than 20k rows will use validation split), or as explicitely specified in the AutoMLConfig class (Validation-split vs. Cross-Validation).

In [6]:
# Split in train/test datasets (Test=10%, Train=90%)

# Split in full sets, if not stratifying
train_df, test_df = train_test_split(data_df, test_size=0.1, random_state=0)

print(train_df.shape)
print(test_df.shape)

train_df.describe()

(535690, 59)
(59522, 59)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5.356900e+05,535690.000000,535690.000000,535690.000000,535690.000000,535690.000000,535690.000000,535690.000000,535690.000000,535690.000000,...,535690.000000,535690.000000,535690.000000,535690.00000,535690.000000,535690.000000,535690.000000,535690.000000,535690.000000,535690.000000
mean,7.437339e+05,0.036433,1.900314,1.359226,4.422926,0.416888,0.404969,0.393683,0.256675,0.164130,...,5.441373,1.441154,2.871545,7.53850,0.122323,0.627996,0.553869,0.287334,0.349058,0.153298
std,4.295112e+05,0.187366,1.983858,0.665157,2.699572,0.493313,1.350742,0.488566,0.436799,0.370394,...,2.333417,1.202829,1.694241,2.74664,0.327658,0.483340,0.497090,0.452519,0.476673,0.360274
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.717152e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.433270e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.00000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.115636e+06,0.000000,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,7.000000,2.000000,4.000000,9.00000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.488027e+06,1.000000,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,13.000000,23.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Train with Azure AutoML automatically searching for the 'best model' (Best algorithms and best hyper-parameters)

### List and select primary metric to drive the AutoML classification problem

In [7]:
from azureml.train import automl
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric

['norm_macro_recall',
 'accuracy',
 'precision_score_weighted',
 'AUC_weighted',
 'average_precision_score_weighted']

## Define AutoML Experiment settings

In [12]:
import logging

# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
automl_settings = {
      "blacklist_models":['LogisticRegression', 'ExtremeRandomTrees', 'RandomForest'], 
      # "whitelist_models": ['LightGBM'],
      "validation_size": 0.1
      # "validation_data": validation_df,  # If you have an explicit validation set
      # "n_cross_validations": 5,
      # "experiment_exit_score": 0.7,
      # "max_cores_per_iteration": -1,
      "enable_voting_ensemble": 'False',
      "enable_stack_ensemble": 'False'
}

from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             primary_metric='AUC_weighted',                           
                             training_data= train_df,  # Pandas DataFrame
                             label_column_name="target",                                                    
                             enable_early_stopping= True,
                             iterations=15,                    
                             experiment_timeout_hours=1,
                             featurization="auto",
                             model_explainability=True,
                             **automl_settings
                             )

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig

## Run Experiment with multiple child runs under the covers

In [ ]:
from azureml.core import Experiment

experiment_name = "SDK_local_porto_seguro_driver_pred"
print(experiment_name)

experiment = Experiment(workspace=ws, 
                        name=experiment_name)

import time
start_time = time.time()

run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s minutes needed for running the whole Remote AutoML Experiment ---' % ((time.time() - start_time)/60))

SDK_local_porto_seguro_driver_pred
Running on local machine
Parent Run ID: AutoML_2c4e24f7-aa40-40e0-afb2-8b5f9fb9d6a4

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------

WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))': /azureml/ExperimentRun/dcid.AutoML_2c4e24f7-aa40-40e0-afb2-8b5f9fb9d6a4_1/outputs/conda_env_v_1_0_0.yml?sv=2019-02-02&sr=b&sig=bHim8LHjQtmRL8RRIC9Xgl%2BC0wvPVS3nG8sX7no7C3g%3D&st=2020-06-06T19%3A19%3A44Z&se=2020-06-07T19%3A29%3A44Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQSUzRA%3D%3D&timeout=30


MaxAbsScaler XGBoostClassifier                 0:03:31       0.6382    0.6382
         2   

WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))': /azureml/ExperimentRun/dcid.AutoML_2c4e24f7-aa40-40e0-afb2-8b5f9fb9d6a4_2/outputs/model.pkl?sv=2019-02-02&sr=b&sig=sF%2F8ev9Rgy%2BFhcm9D5NQlW%2Bu%2FGJbzGuHmhTyKhO7HX0%3D&st=2020-06-06T19%3A21%3A45Z&se=2020-06-07T19%3A31%3A45Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQXdNRFF4T1RRek1EUSUzRA%3D%3D&timeout=30
WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))': /azureml/ExperimentRun/dcid.AutoML_2c4e24f7-aa40-40e0-afb2-8b5f9fb9d6a4_2/outputs/model.pkl?sv=2019-02-02&sr=b&sig=sF%2F8ev9Rgy%2BFhcm9D5NQlW%2Bu%2FGJbzGuHmhTyKhO7HX0%3D&st=2020-06-06T19%3A21%3A45Z&se=2020-06-07T19%3A31%3A45Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdN

MaxAbsScaler LightGBM                          0:02:20       0.6262    0.6382
         3   

...

SparseNormalizer XGBoostClassifier             0:02:21       0.6126    0.6382
         7   

WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))': /azureml/ExperimentRun/dcid.AutoML_2c4e24f7-aa40-40e0-afb2-8b5f9fb9d6a4_7/outputs/model.pkl?sv=2019-02-02&sr=b&sig=HxTyYNTTtEwTe79%2BvIURTUQZ9Mx%2BlbwlPpbrK0kMPhU%3D&st=2020-06-06T19%3A33%3A13Z&se=2020-06-07T19%3A43%3A13Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQXdNRFF4T1RRek1EUSUzRA%3D%3D&timeout=30


MaxAbsScaler SGD                               0:01:57       0.5646    0.6382
         8   MaxAbsScaler SGD                               0:01:56       0.5682    0.6382
         9   StandardScalerWrapper XGBoostClassifier        0:02:21       0.6281    0.6382
        10   

WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))': /azureml/ExperimentRun/dcid.AutoML_2c4e24f7-aa40-40e0-afb2-8b5f9fb9d6a4_10/outputs/model.pkl?sv=2019-02-02&sr=b&sig=8wK6RSDTbYKoQ8ChinV1hN1A%2F8Mj7I0X843bje5R4OM%3D&st=2020-06-06T19%3A39%3A40Z&se=2020-06-07T19%3A49%3A40Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQSUzRA%3D%3D&timeout=30


MaxAbsScaler LightGBM                          0:01:55       0.6232    0.6382
        11   MaxAbsScaler MultinomialNaiveBayes             0:02:14       0.6234    0.6382
        12   MaxAbsScaler SGD                               0:02:07       0.6210    0.6382
        13   

## Explore results with Widget

In [ ]:
# Explore the results of automatic training with a Jupyter widget: https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py
from azureml.widgets import RunDetails
RunDetails(run).show()

## Retrieve the 'Best' Model

In [ ]:
best_run, fitted_model = run.get_output()
print(best_run)
print('--------')
print(fitted_model)

## Make Predictions and calculate metrics

### Prep Test Data: Separate x_test set (feature columns) from y_test (label column) 

In [ ]:
import pandas as pd

x_test_df = test_df.copy()

if 'target' in x_test_df.columns:
    y_test_df = x_test_df.pop('target')

print(test_df.shape)
print(x_test_df.shape)
print(y_test_df.shape)


In [ ]:
y_test_df.describe()

### Make predictions in bulk

In [ ]:
# Try the best model making predictions with the test dataset
y_predictions = fitted_model.predict(x_test_df)

print('30 predictions: ')
print(y_predictions[:30])

### Get all the predictions' probabilities needed to calculate ROC AUC

In [ ]:
class_probabilities = fitted_model.predict_proba(x_test_df)

print(class_probabilities.shape)

print('Some class probabilities...: ')
print(class_probabilities[:3])
print('')
print('Probabilities for class 1:')
print(class_probabilities[:,1])
print('')
print('Probabilities for class 0:')
print(class_probabilities[:,0])

## Evaluate Model

Evaluating performance is an essential task in machine learning. In this case, because this is a classification problem, the data scientist elected to use an AUC - ROC Curve. When we need to check or visualize the performance of the multi - class classification problem, we use AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curve. It is one of the most important evaluation metrics for checking any classification model’s performance.

<img src="https://www.researchgate.net/profile/Oxana_Trifonova/publication/276079439/figure/fig2/AS:614187332034565@1523445079168/An-example-of-ROC-curves-with-good-AUC-09-and-satisfactory-AUC-065-parameters.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 12px; width: 320px; height: 239px;" />

### Calculate the ROC AUC with probabilities vs. the Test Dataset

In [ ]:
print('ROC AUC *method 1*:')
fpr, tpr, thresholds = metrics.roc_curve(y_test_df, class_probabilities[:,1])
metrics.auc(fpr, tpr)


In [ ]:
from sklearn.metrics import roc_auc_score

print('ROC AUC *method 2*:')
print(roc_auc_score(y_test_df, class_probabilities[:,1]))

print('ROC AUC Weighted:')
print(roc_auc_score(y_test_df, class_probabilities[:,1], average='weighted'))


### Calculate the Accuracy with predictions vs. the Test Dataset

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy:')
print(accuracy_score(y_test_df, y_predictions))


## Download model pickle file from the run

In [ ]:
# Download the model .pkl file to local (Using the 'run' object)
best_run.download_file('outputs/model.pkl')

### Load model in memory from .pkl file

In [ ]:
# Load the model into memory
import joblib
fitted_model = joblib.load('model.pkl')
print(fitted_model)